# Linelist generator

This is a simple notebook to generate a linelist for use by `cloudy`.

In [26]:
import numpy as np
from synthesizer.photoionisation import cloudy23

In [27]:

wavelength_type = 'standard'
wavelength_type = 'vacuum'

# the reference line to use
if wavelength_type == 'standard':
    reference_line = 'H 1 4861.32A'
if wavelength_type == 'vacuum':
    reference_line = 'H 1 4862.69A'

# log constrast relative to Hbeta to consider
contrast = 2.0

# minimum wavelength of lines to consider
min_wavelength = 1000

# maximum wavelength of lines to consider
max_wavelength = 25000

### Stellar models

In [28]:

linelist = []

model = r'metallicity_'+wavelength_type
for i in range(4):
    i += 1
    line_ids, blends, wavelengths, intrinsic, emergent = cloudy23.read_lines(rf'{model}/{i}')
    Hbeta = emergent[line_ids==reference_line][0]
    s = (emergent > (Hbeta - contrast)) & (wavelengths<max_wavelength) & (wavelengths>min_wavelength) & (blends == False)
    print(i, np.sum(s))
    linelist += list(line_ids[s])



1 136
2 109
3 72
4 52


### AGN models

In [29]:

model = r'relagn_'+wavelength_type

for i in range(3):
    i += 1
    line_ids, blends, wavelengths, intrinsic, emergent = cloudy23.read_lines(rf'{model}/{i}')
    Hbeta = emergent[line_ids==reference_line][0]
    s = (emergent > (Hbeta - contrast)) & (wavelengths<max_wavelength) & (wavelengths>min_wavelength) & (blends == False)
    print(i, np.sum(s))
    linelist += list(line_ids[s])


1 666
2 397
3 168


In [30]:
linelist = list(set(linelist))
linelist.sort()

# print(linelist)
print(len(linelist))

linelist_ = []

for line in linelist:

    element, ion, wavelength = line.split(' ')

    if len(ion)==1:

        line_ = f'{element} {ion} {wavelength}'
        print(line_)
        linelist_.append(line_)


with open(f'linelist-{wavelength_type}.dat', 'w') as file:
    file.writelines('\n'.join(linelist_) + '\n')

688
Al 2 1670.79A
Al 2 2661.15A
Al 2 2669.95A
Al 3 1854.72A
Al 3 1862.79A
Al 6 2429.13A
Ar 3 7137.76A
Ar 3 7753.24A
Ar 4 2854.50A
Ar 4 2869.06A
Ar 4 4712.58A
Ar 4 4741.45A
Ar 4 7172.68A
Ar 4 7239.77A
Ar 4 7265.33A
Ar 5 2691.85A
Ar 5 6436.90A
Ar 5 7007.76A
C 1 1560.31A
C 1 1560.68A
C 1 1560.71A
C 1 1561.34A
C 1 1561.37A
C 1 1561.44A
C 1 1656.27A
C 1 1656.93A
C 1 1657.01A
C 1 1657.38A
C 1 1657.91A
C 1 1658.12A
C 1 1992.01A
C 1 1993.62A
C 1 2479.31A
C 1 2583.67A
C 1 8729.52A
C 1 9826.82A
C 1 9852.96A
C 2 1036.34A
C 2 1037.02A
C 2 1334.53A
C 2 1335.66A
C 2 1335.71A
C 2 2324.21A
C 2 2325.40A
C 2 2326.11A
C 2 2327.65A
C 2 2328.84A
C 3 1174.93A
C 3 1175.59A
C 3 1906.68A
C 3 1908.73A
C 4 1548.19A
C 4 1550.78A
Ca 2 3934.78A
Ca 2 3969.59A
Ca 2 7293.48A
Ca 2 7325.90A
Ca 2 8500.36A
Ca 2 8544.44A
Ca 2 8664.52A
Ca 5 5310.59A
Ca 5 6088.06A
Ca 8 2.32180m
Cl 2 8581.05A
Cl 2 9126.10A
Cl 3 5519.24A
Cl 3 5539.41A
Cl 4 8047.84A
Co 2 1.01907m
Co 2 1.54742m
Co 2 1470.18A
Co 2 1575.91A
Co 2 1949.74A
Co 2 2019